<a href="https://colab.research.google.com/github/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/blob/main/1%20Data%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dataset Berlin Airbnb Ratings**

https://www.kaggle.com/datasets/thedevastator/berlin-airbnb-ratings-how-hosts-measure-up

Dataset contains 450 thousand comments of 23.5 thousand unique apartments.

We will analyze comments and store aggregated data for apartments.

# **Data Preparation**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install openpyxl -q
import openpyxl
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Comment analisis
from textblob import TextBlob

from datetime import datetime

# missing data
!pip install missingno -q
import missingno as msno

# Set display options
%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
apartments = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/apartments.xlsx', engine='openpyxl')

In [ ]:
comments = pd.read_excel('https://raw.githubusercontent.com/IlyaZutler/Project-3-Berlin-Airbnb-Ratings/main/Comments_full.xlsx', engine='openpyxl')

## Comment Analysis



In [ ]:
# from textblob import TextBlob

def calculate_polarity(comment):
  if pd.notna(comment):
    text = TextBlob(comment)
    return text.sentiment[0]
  else:
    return None

comments['polarity'] = comments['Comments'].apply(calculate_polarity)

In [ ]:
comments_grouped = comments.groupby('Listing ID').agg(
    Polarity_min=('polarity', 'min'),
    Polarity_max=('polarity', 'max'),
    Polarity_median=('polarity', 'median'),
    Polarity_mean=('polarity', 'mean')
)
comments_grouped.head()

,Polarity_min,Polarity_max,Polarity_median,Polarity_mean
Listing ID,,,,
2695,0.27,0.776667,0.464583,0.477044
3176,0.00,1.000000,0.400608,0.400101
7071,0.00,1.000000,0.400000,0.393207
9991,0.00,1.000000,0.424899,0.392466
14325,0.00,1.000000,0.400000,0.408001


In [ ]:
apartments = apartments.merge(comments_grouped, left_on='Listing ID', right_on='Listing ID', how='left')

## Preprocessing data in time format

In [ ]:
# from datetime import datetime

apartments['Host Since'] = pd.to_datetime(apartments['Host Since'])
apartments['First Review'] = pd.to_datetime(apartments['First Review'])
apartments['Last Review'] = pd.to_datetime(apartments['Last Review'])

today = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))

apartments['Host Since Years'] = (today - apartments['Host Since']).dt.days / 365.25
apartments['Last Review Years'] = (today - apartments['Last Review']).dt.days / 365.25
apartments['First Review Years'] = (today - apartments['First Review']).dt.days / 365.25

### Separating numeric and non-numeric data

In [ ]:
apartments['Listing ID'] = apartments['Listing ID'].astype(str)
apartments['Postal Code'] = apartments['Postal Code'].astype(str)

In [ ]:
# import pickle

with open('apartments2.pkl', 'wb') as f:
    pickle.dump(apartments, f)

In [ ]:
# apartments.to_excel('apartments_2.xlsx', index=False)

-------------------end of 1-----------------------